In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/geo/lsoa_land_cover.csv")

In [3]:
res_cols_2018 = [
    "Continuous urban fabric [111] (2018)",
    "Discontinuous urban fabric [112] (2018)"
]

df["residential_pct_2018"] = df[res_cols_2018].sum(axis=1)

output_df = df[["LSOA11CD", "LSOA11NM", "residential_pct_2018"]]

output_df.to_csv("../data/geo/lsoa_residential_percent_2018.csv", index=False)

In [4]:
codes_2021 = pd.read_parquet("../data/london_areas_lookup.parquet", engine='fastparquet')
lookup_mapping = codes_2021.drop_duplicates('LSOA11CD').set_index('LSOA11CD')['LSOA21CD'].to_dict()

df['LSOA21CD'] = df['LSOA11CD'].map(lookup_mapping)

In [5]:
cols_2018 = [col for col in df.columns if '(2018)' in col]
non_res_cols = [col for col in cols_2018 if col not in res_cols_2018]

df["max_non_residential_pct"] = df[non_res_cols].max(axis=1)
df["is_residential_dominant"] = df["residential_pct_2018"] > df["max_non_residential_pct"]
df["residential_advantage"] = df["residential_pct_2018"] - df["max_non_residential_pct"]

london_df = df[df['LSOA21CD'].notna()]

output_df2 = london_df[["LSOA11CD", "LSOA21CD", "LSOA11NM", "residential_pct_2018", "is_residential_dominant", "residential_advantage"]]
output_df2.to_csv("../data/geo/lsoa_residential_classification_2018.csv", index=False)